In [1]:
!pip install --quiet google-api-python-client google-auth google-auth-oauthlib

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [6]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [7]:
from googleapiclient.discovery import build

In [8]:
markdown_text = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
""".strip().splitlines()

In [10]:
from googleapiclient.discovery import build
docs_service = build("docs", "v1", credentials=creds)


In [11]:
doc = docs_service.documents().create(
    body={"title": "Product Team Sync - May 15, 2023"}
).execute()

DOCUMENT_ID = doc["documentId"]
print("Created document:", DOCUMENT_ID)

Created document: 1Z9TDXl5x-n5Wx55HGlRqgJ_tRaDaqzIDqy3Ovf1zjm0


In [12]:
def insert_text(requests, text):
    requests.append({
        "insertText": {
            "location": {"index": 1},
            "text": text
        }
    })

def apply_style(requests, start, end, style):
    requests.append({
        "updateParagraphStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "paragraphStyle": style,
            "fields": ",".join(style.keys())
        }
    })

def apply_text_style(requests, start, end, style):
    requests.append({
        "updateTextStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "textStyle": style,
            "fields": ",".join(style.keys())
        }
    })

In [26]:
import re

requests = []
cursor = 1

for raw_line in markdown_text:
    line = raw_line.rstrip()

    # Strip markdown syntax
    text = line

    if line.startswith("### "):
        text = line[4:]

    elif line.startswith("## "):
        text = line[3:]

    elif line.startswith("# "):
        text = line[2:]

    elif line.strip().startswith("- [ ]"):
        text = line.replace("- [ ]", "", 1).strip()

    elif line.strip().startswith("* "):
        text = line.strip()[2:]

    elif line.strip().startswith("- "):
        text = line.strip()[2:]

    text += "\n"

    # Insert cleaned text
    requests.append({
        "insertText": {
            "location": {"index": cursor},
            "text": text
        }
    })

    start = cursor
    end = cursor + len(text)

    # Apply formatting

    # Headings (longest match first)
    if raw_line.startswith("### "):
        apply_style(requests, start, end, {"namedStyleType": "HEADING_3"})

    elif raw_line.startswith("## "):
        apply_style(requests, start, end, {"namedStyleType": "HEADING_2"})

    elif raw_line.startswith("# "):
        apply_style(requests, start, end, {"namedStyleType": "HEADING_1"})

    # Numbered agenda items should still be headings
    elif re.match(r"\d+\.\s", text):
        apply_style(requests, start, end, {"namedStyleType": "HEADING_3"})

    # Checkboxes
    elif raw_line.strip().startswith("- [ ]"):
        # Reset paragraph style to normal text
        apply_style(requests, start, end, {"namedStyleType": "NORMAL_TEXT"})

        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": "BULLET_CHECKBOX"
            }
        })

        if "@" in raw_line:
            at = raw_line.index("@")
            apply_text_style(
                requests,
                start + at - 6,  # adjust for stripped "- [ ] "
                end,
                {
                    "bold": True,
                    "foregroundColor": {
                        "color": {"rgbColor": {"blue": 1.0}}
                    }
                }
            )

    # Regular bullets
    elif raw_line.strip().startswith(("* ", "- ")):
        # Reset paragraph style to normal text
        apply_style(requests, start, end, {"namedStyleType": "NORMAL_TEXT"})

        requests.append({
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
            }
        })

    # Horizontal rule
    elif raw_line.startswith("---"):
        apply_text_style(requests, start, end, {"italic": True})

    # Footer
    elif raw_line.startswith("Meeting recorded by") or raw_line.startswith("Duration"):
        apply_text_style(
            requests,
            start,
            end,
            {
                "italic": True,
                "foregroundColor": {
                    "color": {"rgbColor": {"red": 0.4}}
                }
            }
        )

    cursor = end

In [27]:
try:
    docs_service.documents().batchUpdate(
        documentId=DOCUMENT_ID,
        body={"requests": requests}
    ).execute()
    print("Document formatted successfully")
except Exception as e:
    raise RuntimeError("Failed to update document") from e

Document formatted successfully
